# Joint SC and FC analysis

We can analyze the brain network using SC and FC. The comparison of the results allows for studying the relationship between structure and function. In other words, does a node with many anatomical connections (SC) also has strong correlations with other regions (FC)?

In [ ]:
import numpy as np
import networkx as nx
import scipy.stats as stt

import sireneta as sna
from sireneta.netmodels import ShuffleLinks, ShuffleWeightsFixedLinks

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.size'] = 16

Let's have a look at an example SC matrix, using the 

In [ ]:
# another example data from HCP with both structural and functional connectomes
SC = np.load('../../Data/ex_SC.npy')

# "renormalize" the weights using log
SC_log = np.log(np.maximum(SC,1))

In [ ]:
plt.figure()
plt.hist(SC.flatten(), bins=100)

plt.figure()
plt.hist(SC.flatten(), bins=100)
plt.axis(ymax=100)

print('number of zero weights in the original SC:', (SC==0).sum())

plt.show()

In [ ]:
plt.figure()
plt.imshow(SC)
plt.title('original SC')
plt.colorbar()
plt.tight_layout()

plt.figure()
plt.imshow(SC_log)
plt.title('log SC')
plt.colorbar()
plt.tight_layout()

plt.show()

Above we see clearly the two hemispheres in the SC matrix, with two sets of indices: from 0 to 49 and then from 50 to 99. The two versions of the SC matrix differ by the scale for the weights, with few extreme values for the original SC as compared to many large values for the log scale (mainly within the same hemisphere, though).

We normalize the weights of both between 0 and 1.

In [ ]:
SC = SC / SC.max()
SC_log = SC_log / SC_log.max()

# number of regions
N = SC.shape[0]

In [ ]:
print(np.linalg.eigvals(SC_log))
print(np.linalg.eigvals(SC_log).max())

In [ ]:
vt = np.linspace(0,20,100)
plt.plot(vt, np.exp(-(1.0/0.8-1.0) * vt))

## Calculate network responses for all 5 propagator models

Here we identify important regions (nodes) and groups of nodes determined by anatomical connections. We first look into the histogram of SC weights to binarize the matrix and create an undirected binary graph.

- check effect of preproc (log)
- check effect of prop model: similarity matrix between prop models for different measures
- discuss equivalent of classical measures when changing prop model
    - strength across time (decide one time to check in particular?)
    - geodesic graph distance, TTP
    - hierarchical community detection (merging over time), spectral gap of random walker Lambiotte

In [ ]:
# normalize connectivity with respect to largest eigenvalue
if False:
    con = SC
    lambda_max = np.linalg.eigvals(SC).max()
    con_norm = SC / lambda_max
else:
    con = SC_log
    lambda_max = np.linalg.eigvals(SC_log).max()
    con_norm = SC_log / lambda_max

# time specifics
dt = 0.1
T = 20.1
td = np.arange(0,T, dtype=int)
tc = np.arange(0,T,dt)
nT = td.size

rdc = sna.Resp_DiscreteCascade(con, tmax=20)
rrw = sna.Resp_RandomWalk(con, tmax=20)
rcc = sna.Resp_ContCascade(con_norm, tmax=20, timestep=dt)
rlc = sna.Resp_LeakyCascade(con_norm, tau=0.8, tmax=20, timestep=dt)
rcd = sna.Resp_ContDiffusion(con_norm, tmax=20, timestep=dt)

# list of responses
ts = [td, td, tc, tc, tc]
rs = [rdc, rrw, rcc, rlc, rcd]
labels = ['dc', 'rw', 'cc', 'lc', 'cd']
n_pm = 5

In [ ]:
plt.figure()
plt.subplot(211)
for i in range(0,3):
    plt.plot(ts[i], sna.GlobalResponse(rs[i]), label=labels[i])
plt.axis(ymin=80, ymax=150)
plt.legend(fontsize=10)
plt.subplot(212)
for i in range(3,5):
    plt.plot(ts[i], sna.GlobalResponse(rs[i]), label=labels[i])
plt.legend(fontsize=10)
plt.show()

In [ ]:
# node responses
inrs = []
onrs = []
for i in range(n_pm):
    inrs.append(sna.NodeResponses(rs[i])[0])
    onrs.append(sna.NodeResponses(rs[i])[1])

aspects = [0.1, 0.1, 1.0, 1.0, 1.0]

In [ ]:
plt.figure(figsize=[10,10])
for i in range(n_pm):
    plt.subplot(n_pm,1,i+1)
    plt.imshow(inrs[i].T, aspect=aspects[i])
    plt.title(labels[i])
    plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[10,10])
for i in range(n_pm):
    plt.subplot(n_pm,1,i+1)
    plt.imshow(onrs[i].T, aspect=aspects[i])
    plt.title(labels[i])
    plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[10,10])
for i in range(n_pm):
    plt.subplot(n_pm,1,i+1)
    for j in range(20):
        plt.plot(inrs[i][:,j], onrs[i][:,j])
    plt.title(labels[i])
plt.tight_layout()
plt.show()

In [ ]:
i = 0
j = 1

plt.figure()
for t in td:
    plt.scatter(inrs[i][t,:], inrs[j][t,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.figure()
plt.scatter(onrs[i][td,:], onrs[j][td,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.show()

In [ ]:
i = 0
j = 2

plt.figure()
plt.scatter(inrs[i][td,:], inrs[j][tdt,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.figure()
plt.scatter(onrs[i][td,:], onrs[j][tdt,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.show()

In [ ]:
i = 0
j = 3

plt.figure()
plt.scatter(inrs[i][td,:] , inrs[j][tdt,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.figure()
plt.scatter(onrs[i][td,:], onrs[j][tdt,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.show()

In [ ]:
i = 0
j = 4

plt.figure()
plt.scatter(inrs[i][td,:], inrs[j][tdt,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.figure()
plt.scatter(onrs[i][td,:], onrs[j][tdt,:])
plt.xlabel(labels[i])
plt.ylabel(labels[j])

plt.show()

In [ ]:
# similarity between node responses
dist_node = np.zeros([n_pm,n_pm,nT])

tdt = []
for t in td:
    # equivalent continuous time
    tdt.append(int(t/dt))
    
tts = [td, td, tdt, tdt, tdt]
    
for t in td:
    for i in range(n_pm):
        for j in range(n_pm):
            dist_node[i,j,t] = 1.0 - stt.pearsonr(inrs[i][tts[i][t],:], inrs[j][tts[j][t],:])[0]
            
plt.figure(figsize=[10,10])
for it, t in enumerate(range(1,nT,4)):
    plt.subplot(5,1,it+1)
    plt.imshow(dist_node[:,:,t], vmin=0.0, vmax=1.0, cmap='hot')
    plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
# similarity between node responses
dist_node = np.zeros([n_pm,n_pm,nT])

tdt = []
for t in td:
    # equivalent continuous time
    tdt.append(int(t/dt))
    
tts = [td, td, tdt, tdt, tdt]
    
for t in td:
    for i in range(n_pm):
        for j in range(n_pm):
            dist_node[i,j,t] = 1.0 - stt.pearsonr(onrs[i][tts[i][t],:], onrs[j][tts[j][t],:])[0]
            
plt.figure(figsize=[10,10])
for it, t in enumerate(range(1,nT,4)):
    plt.subplot(5,1,it+1)
    plt.imshow(dist_node[:,:,t], vmin=0.0, vmax=1.0, cmap='hot')
    plt.colorbar()
plt.tight_layout()
plt.show()

# TTP

- for DC, we choose 1 as a threshold
- for 

# Comparison with surrogates

using `ShuffleWeightsFixedLinks` and `ShuffleLinks`

In [ ]:
shuf1_con = ShuffleWeightsFixedLinks(con)
shuf1_con_norm = shuf1_con / np.real(np.linalg.eigvals(shuf1_con)).max()
shuf1_rlc = sna.Resp_LeakyCascade(shuf1_con_norm, tau=0.8, tmax=20, timestep=dt)
shuf1_inrlc, shuf1_onrlc = sna.NodeResponses(shuf1_rlc)

shuf2_con = ShuffleLinks(con)
shuf2_con_norm = shuf2_con / np.real(np.linalg.eigvals(shuf2_con)).max()
shuf2_rlc = sna.Resp_LeakyCascade(shuf1_con_norm, tau=0.8, tmax=20, timestep=dt)
shuf2_inrlc, shuf2_onrlc = sna.NodeResponses(shuf2_rlc)

In [ ]:
plt.figure(figsize=[10,10])
plt.subplot(3,1,1)
plt.plot(tc, inrs[3])
plt.title('lc')
plt.subplot(3,1,2)
plt.plot(tc, shuf1_inrlc)
plt.title('lc shuf fixed links')
plt.subplot(3,1,3)
plt.plot(tc, shuf2_inrlc)
plt.title('lc shuf full')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[10,10])
plt.subplot(3,1,1)
plt.plot(inrs[3], onrs[3])
plt.title('lc')
plt.subplot(3,1,2)
plt.plot(shuf1_inrlc, shuf1_onrlc)
plt.title('lc shuf fixed links')
plt.subplot(3,1,3)
plt.plot(shuf2_inrlc, shuf2_onrlc)
plt.title('lc shuf full')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[10,10])
plt.subplot(2,1,1)
plt.imshow(inrs[3].T-shuf1_inrlc.T, aspect=aspects[i])
plt.title('lc - shuf fixed links')
plt.subplot(2,1,2)
plt.imshow(inrs[3].T-shuf2_inrlc.T, aspect=aspects[i])
plt.title('lc - shuf full')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[10,10])
plt.subplot(2,1,1)
plt.imshow(onrs[3].T-shuf1_onrlc.T, aspect=aspects[i])
plt.title('lc - shuf fixed links')
plt.subplot(2,1,2)
plt.imshow(onrs[3].T-shuf2_onrlc.T, aspect=aspects[i])
plt.title('lc - shuf full')
plt.tight_layout()
plt.show()

# OLD

In [ ]:
plt.figure()
plt.hist(SC_log.flatten())
plt.xlabel('SC value')
plt.ylabel('con coount')
plt.tight_layout()

SC_bin = np.array(SC>0.01, dtype=int)
#SC_bin = np.array(SC_log>0.5, dtype=int)
print(SC_bin)
print(SC_bin.diagonal())

In [ ]:
G = nx.from_numpy_array(SC_bin)
nx.draw_circular(G)

print('density:', nx.density(G))

In [ ]:
plt.figure()
plt.imshow(SC_bin)
plt.title('binarized SC')
plt.colorbar()
plt.tight_layout()

In [ ]:
hist_deg = nx.degree_histogram(G)

plt.figure()
plt.bar(range(len(hist_deg)), hist_deg)
plt.xticks(range(0,len(hist_deg),5))
plt.xlabel('degree')
plt.ylabel('node count')
plt.show()

We can compare several centrality measures (node metrics) to see if they identify similar nodes as important nodes.

In [ ]:
dict_dc = nx.degree_centrality(G)
dict_ec = nx.eigenvector_centrality(G)
dict_bc = nx.betweenness_centrality(G)

# keep results for comparison later
ec_SC = np.array([v for v in nx.eigenvector_centrality(G).values()])

bins = np.linspace(0.0,1.0,40)
plt.figure()
plt.hist(dict_dc.values(), bins=bins, histtype='step', label='deg')
plt.hist(dict_ec.values(), bins=bins, histtype='step', label='eig')
plt.hist(dict_bc.values(), bins=bins, histtype='step', label='bet')
plt.xlabel('centrality value')
plt.ylabel('node count')
plt.legend()
plt.tight_layout()

In [ ]:
plt.figure()
plt.scatter(dict_dc.values(), dict_ec.values())
plt.xlabel('degree centrality')
plt.ylabel('eigenvector centrality')
plt.show()

We can see with the scatter plot that the important nodes for degree centrality are globally the same as those for eigenvector centrality. What about betweenness centrality?

In [ ]:
plt.figure()
plt.hist(dict_ec.values(), bins=40, histtype='step', label='eig')
plt.xlabel('eig centrality value')
plt.ylabel('node count')
plt.legend()
plt.tight_layout()

plt.figure(figsize=[12,6])
plt.subplot(1,3,1)
plt.imshow(SC)
plt.subplot(1,3,2)
plt.imshow(SC_bin)
plt.subplot(1,3,3)
v_ec_SC = np.zeros([N,1])
for i, ec in enumerate(dict_ec.values()):
    if ec>0.11:
        v_ec_SC[i] = 1
plt.imshow(v_ec_SC, aspect=0.01)
plt.show()

The right panel on the bottom plot corresponds to a binary vector with important ndoes in yellow (value equal to 1). In this way we obtain a list of important nodes in the networkv, the threshold being determined from the histogram.

What about the structure in terms of group of nodes? Let's check the core?

In [ ]:
pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos=pos)
nx.draw_networkx_labels(G, pos=pos)

G_core = nx.k_core(G)
nx.draw(G_core, pos=pos, node_color='red')

plt.show()

In [ ]:
plt.figure(figsize=[12,6])
plt.subplot(1,3,1)
plt.imshow(SC)
plt.subplot(1,3,2)
plt.imshow(SC_bin)
plt.subplot(1,3,3)
v_core = np.zeros([N,1])
for i in G_core:
    v_core[i] = 1
plt.imshow(v_core, aspect=0.01)
plt.show()

The core indicates that almost all regions are strongly connected, without a division between the two hemispheres. Let's have a look at communities from the binary matrix.

In [ ]:
list_com = nx.community.greedy_modularity_communities(G)
for i, com in enumerate(list_com):
    print(f'{i}-th community:', sorted(com))

In [ ]:
plt.figure(figsize=[18,6])
plt.subplot(1,3,1)
plt.imshow(SC)
plt.subplot(1,3,2)
plt.imshow(SC_bin)
plt.subplot(1,3,3)
v_comm_SC = np.zeros([N,N])
for c in range(len(list_com)):
    for i in list_com[c]:
        for j in list_com[c]:
            # check if nodes i and j in same community for SC then FC
            if not i==j:
                v_comm_SC[i,j] = c+1
plt.imshow(v_comm_SC, vmin=0.0, cmap='tab20')
plt.colorbar()

plt.show()